# 📈 Equity Valuation Demo

This notebook demonstrates multiple equity valuation models in Python using `yfinance`.  
It supports automatic data fetching, model calculation, and export of results in CSV and HTML (with colour-coded valuation notes).

---

## 🧮 Implemented & Supported Models

| Model | Status | Data Source / Calculation | Formula |
|---|---|---|---|
| **DCF (Discounted Cash Flow)** | ✅ Implemented | Free cash flow from cashflow statement | See formula below |
| **DDM (Dividend Discount Model)** | ✅ Implemented | Dividend history or EPS × payout ratio | Two-stage growth |
| **Graham Number** | ✅ Implemented | EPS & Book Value per Share | \( \sqrt{22.5 \times EPS \times BVPS} \) |
| **Buffett Fair Value Formula** | ✅ Implemented | EPS & growth rate | \( EPS \times (8.5 + 2 \times g) \) |
| **Relative Valuation (P/E, P/B)** | ⚠️ Data pulled only | `trailingPE` & `priceToBook` from `yfinance` | Direct comparison |
| **Residual Income Model (ROE)** | ⚠️ Data pulled only | `returnOnEquity`, BVPS, cost of equity | \( BV + \sum \frac{RI_t}{(1+r)^t} \) |
| **EV/EBITDA Multiple** | ⚠️ Data pulled only | EV & EBITDA | \( \frac{EV}{EBITDA} \) |

✅ = fully calculated in code • ⚠️ = data retrieved; extend as needed

---

## ⚙️ Workflow
1. **Load configuration and tickers** – `config.json`, `tickers.txt`  
2. **Fetch financial data** – cash flow, EPS, BVPS, dividends, P/E, P/B, ROE, EV/EBITDA  
3. **Run valuation models** – DCF, DDM, Graham, Buffett  
4. **Aggregate** – median fair value, Upside %, valuation flag  
5. **Export** – `output/results.csv`, `output/report.html`

---

## 📐 Models & Formulae

### 1. Discounted Cash Flow (DCF)

Assumptions:
- Current free cash flow \( FCF_0 \)
- Annual growth rate \( g \)
- Discount rate \( r \)
- Terminal growth rate \( g_t \)
- Projection horizon \( N \)

# Corporate Valuation Formula

## Firm Value Calculation Formula

$$
\text{Firm Value} = \sum_{t=1}^{N} \frac{FCF_0 \cdot (1+g)^t}{(1+r)^t} + \frac{FCF_N \cdot (1+g_t)}{(r - g_t)} \cdot \frac{1}{(1+r)^N}
$$

where: `FCF_N = FCF_0 × (1+g)^N`

## Fair Value per Share

$$
\text{Fair Value per Share} = \frac{\text{Firm Value} + \text{Cash} - \text{Debt}}{\text{Shares Outstanding}}
$$

## Variables

- \( FCF_0 \): base free cash flow
- \( g \): growth rate
- \( r \): discount rate (WACC)
- \( N \): projection years
- \( g_t \): terminal growth rate

---

### 2. Dividend Discount Model (Two-Stage DDM)

Assumptions:
- Current dividend \( D_0 \)
- First-stage growth \( g_1 \) (for \( n \) years)
- Terminal (second-stage) growth \( g_2 \)
- Discount rate \( r \)

Formula:

$$\text{Value} = \sum_{t=1}^{n} \frac{D_0 \times (1+g_1)^t}{(1+r)^t} + \frac{D_n \times (1+g_2)}{(r - g_2)} \times \frac{1}{(1+r)^n}$$

where:

$$D_n = D_0 \times (1+g_1)^n$$

---

### 3. Graham Number

Benjamin Graham’s conservative fair-value formula:

$$
\text{Graham Number} = \sqrt{22.5 \cdot EPS \cdot BVPS}
$$

- **EPS**: earnings per share
- **BVPS**: book value per share

---

### 4. Buffett Approximation

Buffett Fair Value Formula:

$$
\text{Fair Price} = EPS \times (8.5 + 2 \times g)
$$

- \( g \): growth rate (%)

---

> Notes (implementation details in this notebook):
> - DCF requires your own cash flow forecasting; here we approximate using the latest FCF and a growth assumption.
> - DDM takes dividend history when available; otherwise uses EPS × payout ratio as a proxy for \( D_0 \).
> - Buffett method compares your fair price to current market price.
> - Relative valuation (P/E, P/B), ROE-based residual income, and EV/EBITDA are retrieved for comparison; extend formulas as needed.

# 股票估值模型實作說明

這份 Notebook 使用 `yfinance` 套件自動抓取美股公司基本資料，並計算以下 7 種估值模型的關鍵指標：

1. **DCF 模型（需自建現金流預測）**：未在此直接實作，但可使用 `freeCashflow` 欄位作為基礎。
2. **股利折現模型（DDM）**：抓取 `dividendRate`，可進一步用 D = D1 / (r - g) 計算。
3. **巴菲特合理價格估值法**：需使用你的主觀估值與當前市值比較。
4. **相對估值法（P/E, P/B）**：透過 `trailingPE` 與 `priceToBook` 直接比較。
5. **格雷厄姆數（Graham Number）**：透過公式 √(22.5 × EPS × BVPS) 計算合理價值。
6. **剩餘收益模型（ROE 模型）**：取 `returnOnEquity`，可進一步導入帳面價值與資本成本做精算。
7. **EV/EBITDA 評估法**：計算企業價值（EV）與 EBITDA 的比例，衡量企業併購合理性。

這些指標可幫助你快速評估股票是否被高估或低估，並作為進一步選股分析的基礎。


 # 1.📘 DCF 模型（Discounted Cash Flow）折現現金流估值法

# Intrinsic Valuation of a Company using Discounted Cash Flows (To Firm)


## 🎯 DCF 適用股票類型建議

## 🎯 DCF 適用股票類型建議（含實例）

| 適合使用 DCF 模型的公司類型           | 原因                      | 代表例子                                |
|----------------------------------------|---------------------------|-----------------------------------------|
| 有穩定自由現金流                      | 現金流是 DCF 核心         | **AAPL**、**MSFT**、**PG**、**KO**、**XOM**、**WMT**、**JPM** |
| 非高配息但持續營運的企業              | 無股利也可用              | **AMZN**、**META**、**NVDA**、**AVGO**、**AXP** |
| 成長放緩中的大型科技股                | 成長可預測，FCF 穩定可估  | **INTC**、**CRM**、**GOOGL**、**TSM** |


$$
\begin{aligned}
\sum_{t=1}^N \frac{Cash Flow To Firm_{t}}{(1+r)^{t}}
\end{aligned}
$$
Cash Flow to Firm = Expected Cash flow to firm in period t \
r = Weighted Cost of Capital (WACC) = Weight of Equity * Cost of Equity + Weight of Debt * Cost of Debt \
N = Life of the asset

# Key Assumptions when using Cash Flow to Firm
Cash flows considered are cash flows from assets, prior to any debt payments but after firm has reinvested to create growth assets\
Discount rate reflects the cost of raising both debt and equity financing, in proportion to their use
### NOTE: This is DIFFERENT from the DCF valuation of Cash flow to equity (Detailed in a future video)


In [22]:
import yfinance as yf
import pandas as pd
import numpy as np
import time

def dcf_valuation(ticker, fcf_growth_rate=0.06, discount_rate=0.09, projection_years=5, terminal_growth=0.025):
    stock = yf.Ticker(ticker)
    
    try:
        cashflow = stock.cashflow
        if cashflow.empty:
            return {"Ticker": ticker, "Fair Value": "N/A", "Note": "Cashflow empty"}

        # 正確的自由現金流組成項目
        cf_index = cashflow.index.tolist()
        if "Operating Cash Flow" in cf_index and "Capital Expenditure" in cf_index:
            op_cashflow = cashflow.loc["Operating Cash Flow"].dropna().iloc[-1]
            capex = cashflow.loc["Capital Expenditure"].dropna().iloc[-1]
        else:
            return {"Ticker": ticker, "Fair Value": "N/A", "Note": "Missing FCF components"}

        fcf = float(op_cashflow + capex)
        if fcf <= 0:
            return {"Ticker": ticker, "Fair Value": "N/A", "Note": "FCF <= 0"}

        # 預估未來 FCF 並折現
        fcf_list = [
            fcf * ((1 + fcf_growth_rate) ** year) / ((1 + discount_rate) ** year)
            for year in range(1, projection_years + 1)
        ]

        # 終值與折現
        terminal_fcf = fcf * ((1 + fcf_growth_rate) ** projection_years)
        terminal_value = terminal_fcf * (1 + terminal_growth) / (discount_rate - terminal_growth)
        terminal_value_discounted = terminal_value / ((1 + discount_rate) ** projection_years)

        enterprise_value = sum(fcf_list) + terminal_value_discounted

        # 股權價值估算
        info = stock.info
        debt = info.get("totalDebt", 0) or 0
        cash = info.get("totalCash", 0) or 0
        shares = info.get("sharesOutstanding", 1)
        current_price = info.get("currentPrice", np.nan)

        equity_value = enterprise_value + cash - debt
        fair_value_per_share = equity_value / shares
        margin_of_safety = (fair_value_per_share - current_price) / fair_value_per_share * 100 if current_price else np.nan

        note = "Undervalued" if fair_value_per_share > current_price else "Overvalued"

        return {
            "Ticker": ticker,
            "Current Price ($)": round(current_price, 2),
            "Fair Value per Share ($)": round(fair_value_per_share, 2),
            "Margin of Safety (%)": round(margin_of_safety, 2),
            "Current FCF (M)": round(fcf / 1e6, 2),
            "Note": note
        }

    except Exception as e:
        return {"Ticker": ticker, "Fair Value": "N/A", "Note": str(e)}


tickers =  ["AAPL", "MSFT", "PG", "KO", "JNJ", "AXP", "JPM", "WMT", "XOM", "GOOG", "TSM"]
results = []

for tkr in tickers:
    result = dcf_valuation(tkr)
    results.append(result)
    time.sleep(1) 

# 整理成表格
df_dcf = pd.DataFrame(results)
df_dcf = df_dcf[df_dcf["Note"].isin(["Undervalued", "Overvalued"])]  # 排除 error 結果
df_dcf = df_dcf.sort_values(by="Margin of Safety (%)", ascending=False).reset_index(drop=True)

# 顯示結果
pd.set_option('display.float_format', '${:,.2f}'.format)
df_dcf= df_dcf.drop(columns=["Fair Value"], errors="ignore")
df_dcf

,Ticker,Current Price ($),Fair Value per Share ($),Margin of Safety (%),Current FCF (M),Note
0,TSM,$194.76,"$1,258.74",$84.53,"$262,724.30",Undervalued
1,GOOG,$166.81,$237.10,$29.64,"$67,012.00",Undervalued
2,XOM,$108.48,$148.48,$26.94,"$36,053.00",Undervalued
3,AXP,$300.10,$341.28,$12.07,"$13,095.00",Undervalued
4,JNJ,$146.36,$144.82,$-1.06,"$19,758.00",Overvalued
5,PG,$158.02,$111.08,$-42.26,"$15,584.00",Overvalued
6,KO,$69.16,$39.50,$-75.07,"$11,258.00",Overvalued
7,AAPL,$212.33,$110.67,$-91.86,"$92,953.00",Overvalued
8,MSFT,$452.94,$134.88,$-235.80,"$56,118.00",Overvalued
9,WMT,$96.83,$18.72,$-417.21,"$11,075.00",Overvalued


# 2.Dividend Discount Model（DDM）股利折現模型 (適合穩定公司)

## 📘 Two-Stage Dividend Discount Model (DDM) 自動化估值

自動為一組股票進行 **兩段式股利折現模型估值（Two-Stage DDM）**，以評估其是否低估或高估。

---

### 🔧 模型原理簡述

兩段式 DDM 假設公司股利未來將：

1. **前 n 年穩定以 g 成長**
2. **n 年後進入穩定期，股利以同樣 g 永續成長**

其估值公式如下：

$$
P_0 = \sum_{t=1}^{n} \frac{D_t}{(1 + r)^t} + \frac{D_{n+1}}{(r - g)} \cdot \frac{1}{(1 + r)^n}
$$

其中：

- \( D_0 \)：目前股利（從 `dividendRate` 取得）
- \( r \)：折現率（用 CAPM 計算：\( r = r_f + \beta (r_m - r_f) \)）
- \( g \)：股利成長率（預設為分析師預估值 `earningsGrowth`，缺失時 fallback 為 4%）
- \( n \)：高成長期長度（預設為 5 年）

---

### 📌 自動計算說明

| 元素             | 來源                       | 備註                         |
|------------------|----------------------------|------------------------------|
| `dividendRate`   | `info["dividendRate"]`     | 年股利，為 D0               |
| `beta`           | `info["beta"]`             | 股價對市場波動敏感度        |
| `earningsGrowth` | `info["earningsGrowth"]`   | 預估成長率，為 g 的來源     |
| `currentPrice`   | `info["currentPrice"]`     | 現價，用於計算安全邊際 (%) |

---

### ⚠️ 邏輯修正與安全保護

- 若股利為 0，則 `Note = "No dividend"`
- 若 `g >= r`，則強制調整為 `g = r - 0.005`，避免終值分母為 0 導致爆炸
- 計算 Margin of Safety（安全邊際）：

$$
\text{Margin of Safety} = \frac{\text{Fair Value} - \text{Price}}{\text{Fair Value}} \times 100\%
$$

---



In [23]:
import yfinance as yf
import pandas as pd
import numpy as np

def ddm_two_stage_auto(ticker, rf=0.045, rm_rf=0.055, n=5, g_default=0.04):
    stock = yf.Ticker(ticker)
    info = stock.info

    try:
        D0 = info.get("dividendRate")
        beta = info.get("beta", 1)
        price = info.get("currentPrice", np.nan)

        # 跳過無股利公司
        if D0 is None or D0 == 0:
            return {"Ticker": ticker, "Fair Value ($)": "N/A", "Note": "No dividend"}

        # Cost of Equity
        r = rf + beta * rm_rf

        
        # 成長率 g 來源穩定，來自分析師預估
        g = info.get("earningsGrowth")
        if g is None or g <= 0:
            g = g_default
        g = min(max(g, 0.02), 0.08)
        
        # 🚨 修正：防止 g ≥ r（終值公式會失效）
        if g >= r:
            g = r - 0.005  # 強制小於 r 一點點
        
    
        
        # DDM 計算
        D_list = [D0 * (1 + g) ** t / (1 + r) ** t for t in range(1, n + 1)]
        terminal_D = D0 * (1 + g) ** n
        terminal_value = terminal_D / (r - g)
        present_terminal = terminal_value / ((1 + r) ** n)
        fair_value = sum(D_list) + present_terminal

        margin = (fair_value - price) / fair_value * 100 if price else np.nan
        note = "Undervalued" if fair_value > price else "Overvalued"

        return {
            "Ticker": ticker,
            "Dividend ($)": round(D0, 2),
            "Beta": round(beta, 2),
            "r (COE)": round(r, 4),
            "g (Growth)": round(g, 4),
            "Fair Value ($)": round(fair_value, 2),
            "Current Price ($)": round(price, 2),
            "Margin of Safety (%)": round(margin, 2),
            "Note": note
        }

    except Exception as e:
        return {"Ticker": ticker, "Fair Value ($)": "N/A", "Note": str(e)}


tickers = ["KO", "JNJ", "PG", "T", "XOM", "AXP", "JPM", "WMT", "TMUS"]

results = [ddm_two_stage_auto(tkr) for tkr in tickers]
df_ddm = pd.DataFrame(results)

# 排除錯誤資料並依安全邊際排序
df_ddm = pd.DataFrame(results).sort_values(by="Margin of Safety (%)", ascending=False).reset_index(drop=True)

pd.set_option('display.float_format', '${:,.2f}'.format)
df_ddm


,Ticker,Dividend ($),Beta,r (COE),g (Growth),Fair Value ($),Current Price ($),Margin of Safety (%),Note
0,T,$1.11,$0.61,$0.08,$0.07,$222.38,$26.44,$88.11,Undervalued
1,JNJ,$5.20,$0.41,$0.07,$0.06,"$1,041.51",$146.36,$85.95,Undervalued
2,TMUS,$3.52,$0.68,$0.08,$0.08,"$1,435.10",$235.29,$83.60,Undervalued
3,XOM,$3.96,$0.47,$0.07,$0.04,$127.93,$108.48,$15.20,Undervalued
4,KO,$2.04,$0.47,$0.07,$0.04,$78.66,$69.16,$12.08,Undervalued
5,JPM,$5.60,$1.07,$0.10,$0.08,$237.83,$265.64,$-11.69,Overvalued
6,PG,$4.23,$0.41,$0.07,$0.02,$89.22,$158.02,$-77.12,Overvalued
7,AXP,$3.28,$1.24,$0.11,$0.08,$100.07,$300.10,$-199.89,Overvalued
8,WMT,$0.94,$0.70,$0.08,$0.04,$21.80,$96.83,$-344.24,Overvalued


# 3. 巴菲特合理價格估值法

## 📘 股票估值方法整合（DCF + 巴菲特模型）

本節介紹兩種估值模型，並著重於 **巴菲特合理價格估值法** 的應用：

---

### ✅ 模型對照總覽

| 模型         | 適用情境                  | 關鍵變數        | 適合公司類型           |
|--------------|---------------------------|------------------|------------------------|
| Buffett 模型 | 預估 EPS 複利成長後折現價值 | EPS、ROE、配息率、PE、r | 高 ROE、長期 EPS 穩定企業 |

---

## 📈 巴菲特合理價格估值法（以 ROE 為核心）

根據巴菲特的投資邏輯與現有資料，我們使用以下方式計算未來十年後的內在價值：

---

### 1. 預估年化成長率

\[
g = \text{ROE} \times (1 - \text{配息率})
\]

- EPS 若全部保留，成長率可達 ROE
- 有配息時，實際成長率會降低

---

### 2. 推估 10 年後每股盈餘（EPS）

\[
EPS_{10} = EPS_{0} \cdot (1 + g)^{10}
\]

---

### 3. 合理市盈率（PE）設定對照表

| ROE 區間     | 合理 PE 值 |
|--------------|-------------|
| ROE ≥ 30%    | 20          |
| ROE ≥ 20%    | 15          |
| ROE ≥ 10%    | 10          |
| ROE < 10%    | 8 或更低     |

---

### 4. 預估 10 年後合理股價

$$
\text{合理價值}_{10年後} = EPS_{10} \cdot PE
$$

---

### 5. 折現至今日合理價值（使用 CAPM 動態 r）

$$
r = r_f + \beta \cdot (r_m - r_f)
$$

\[
\text{今日合理價格} = \frac{\text{合理價值}_{10年後}}{(1 + r)^{10}}
\]

- \( r_f \)：無風險利率，預設 4.5%
- \( r_m - r_f \)：市場風險溢酬，預設 5.5%
- \( \beta \)：來自 Yahoo Finance，每家公司不同

---

## 🧠 為什麼這方法特別適合巴菲特風格？

- 強調公司 **內部報酬率（ROE）**
- 關注企業 **留存盈餘再投資能力**
- 重視 **10 年以上的複利價值**

---

## ✅ 適合使用此估值法的公司類型

| 條件                 | 原因                         | 範例股票                 |
|----------------------|------------------------------|--------------------------|
| 長期 ROE 穩定且高    | 可推估穩定複利成長           | AAPL、MSFT、AVGO         |
| EPS 過去 5 年無虧損  | 成長可計算、模型才有意義     | KO、PG、JNJ、ADP         |
| 成熟期科技股         | 高 ROE + 穩定配息政策        | TXN、INTC、TSM           |
| 配息適中（20%–50%）  | 保留盈餘成長潛力仍足夠       | JPM、WMT、AXP            |

---

## ❌ 不適合使用的公司類型

| 類型             | 為何不適合？                  | 範例                    |
|------------------|-------------------------------|-------------------------|
| 初創公司 / 無 EPS | EPS 不穩或為負                | UBER、PLTR、SNOW       |
| ROE 波動極大      | 無法穩定預估成長率 g          | TSLA（早期）、NFLX     |
| 成長股估值過高    | 雖有 EPS，但本益比常過高失真 | NVDA、SHOP、AMD        |

---

## 📊 評估結果指標

| 指標                     | 說明                                      |
|--------------------------|-------------------------------------------|
| `Buffett Fair Price`     | 依公司 ROE × 盈餘保留推估 10 年價值折現後所得 |
| `r (COE)`                | 每家公司 beta 動態折現率（CAPM）          |
| `Undervalued (Buffett)`  | 現價是否低於模型推估的合理價格             |

---

## 📌 使用建議

- 最適合：**價值投資者長期持有**
- 可搭配：PEG、DCF、現金流殖利率交叉確認
- 對成長型公司：建議改用情境分析或 PEG 模型

---


In [24]:
import yfinance as yf
import numpy as np
import pandas as pd

# CAPM 計算動態折現率
def get_discount_rate(info, rf=0.045, market_premium=0.055):
    beta = info.get("beta", 1.0)
    return rf + beta * market_premium

# Buffett 模型（動態折現率 + 真實配息率）
def buffett_fair_price(eps, roe, payout_ratio, discount_rate, years=10):
    if eps is None or roe is None:
        return None
    # 修正：ROE 必須除以 100
    g = (roe / 100) * (1 - payout_ratio)
    eps_future = eps * ((1 + g) ** years)
    pe = 20 if roe >= 30 else 15 if roe >= 20 else 10 if roe >= 10 else 8
    fair_price = eps_future * pe / ((1 + discount_rate) ** years)
    return fair_price


# 股票清單
tickers = [
    "AAPL",   # Apple - 高 ROE，EPS 穩定
    "MSFT",   # Microsoft - 高 ROE，現金流強
    "KO",     # Coca-Cola - 巴菲特愛股，EPS 穩定
    "PG",     # Procter & Gamble - 長期穩健
    "JNJ",    # Johnson & Johnson - 醫療龍頭，ROE 穩定
    "WMT",    # Walmart - 零售穩健 EPS 成長
    "JPM",    # JPMorgan Chase - 銀行龍頭，EPS 穩定
    "AXP",    # American Express - 巴菲特長期持有
    "TSM",    # 台積電 ADR - 穩定 EPS 與 ROE
]


results = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    info = stock.info

    # 折現率
    discount_rate = get_discount_rate(info)

    # 取得估值參數
    eps = info.get("trailingEps")
    roe = info.get("returnOnEquity")
    price = info.get("currentPrice")
    dividend = info.get("dividendRate")

    # 真實配息比率（若無配息則為 0）
    payout_ratio = max(0, min(1, dividend / eps)) if eps and dividend else 0

    # 巴菲特估值
    buffett_val = buffett_fair_price(eps, roe, payout_ratio, discount_rate=discount_rate)
    buffett_undervalued = (buffett_val is not None and price is not None and price < buffett_val)

    # 結果儲存
    results.append({
        "Ticker": ticker,
        "Current Price": round(price, 2) if price else "N/A",
        "EPS": round(eps, 2) if eps else "N/A",
        "ROE (%)": round(roe * 100, 2) if roe else "N/A",
        "Payout Ratio": round(payout_ratio, 2),
        "r (COE)": round(discount_rate, 4),
        "Margin of Safety (%)": round((buffett_val - price) / buffett_val * 100, 2) if buffett_val else "N/A",
        "Buffett Fair Price ($)": round(buffett_val, 2) if buffett_val else "N/A",
        "Undervalued (Buffett)": "Yes" if buffett_undervalued else "No"
    })

# 整理成表格
df_buffett = pd.DataFrame(results)
df_buffett = df_buffett.sort_values("Undervalued (Buffett)", ascending=False).reset_index(drop=True)
pd.set_option('display.float_format', '${:,.2f}'.format)
df_buffett

,Ticker,Current Price,EPS,ROE (%),Payout Ratio,r (COE),Margin of Safety (%),Buffett Fair Price ($),Undervalued (Buffett)
0,AAPL,$212.33,$6.41,$138.01,$0.16,$0.12,$-997.42,$19.35,No
1,MSFT,$452.94,$12.95,$33.61,$0.26,$0.10,$-997.76,$41.26,No
2,KO,$69.16,$2.49,$38.78,$0.82,$0.07,$-585.02,$10.10,No
3,PG,$158.02,$6.30,$30.29,$0.67,$0.07,$-497.13,$26.46,No
4,JNJ,$146.36,$8.99,$29.45,$0.58,$0.07,$-286.63,$37.86,No
5,WMT,$96.83,$2.41,$21.41,$0.39,$0.08,"$-1,004.85",$8.76,No
6,JPM,$265.64,$20.37,$17.35,$0.27,$0.10,$-331.94,$61.50,No
7,AXP,$300.10,$14.31,$34.27,$0.23,$0.11,$-645.75,$40.24,No
8,TSM,$194.76,$8.20,$31.66,$0.33,$0.12,$-784.16,$22.03,No


# 4. 相對估值模型（P/E, P/B）
---

### 🧠 適用對象：

- 快速評估公司目前的估值是否高於或低於產業平均
- 適用於已有明確盈餘或帳面價值的公司（如大型金融、製造、科技股）

---

### 📌 模型簡介：

相對估值法不是估算公司內在價值，而是**將某家公司的估值倍數與其產業平均或可比公司進行比較**。  
常見指標包括：

#### 🔢 本益比（P/E）— 價格對盈餘的比例：

$$
\text{P/E} = \frac{\text{每股市價}}{\text{每股盈餘 (EPS)}}
$$

- EPS 通常取過去 12 個月（Trailing Twelve Months）
- P/E 越高，代表市場預期成長越高，反之可能被低估

#### 🔢 股價淨值比（P/B）— 價格對帳面淨值的比例：

$$
\text{P/B} = \frac{\text{每股市價}}{\text{每股淨值 (Book Value per Share)}}
$$

- 常用於資產為主的產業，如銀行、保險、不動產
- 若 P/B < 1，表示股價低於帳面價值，可能為低估

---

### 📊 分析邏輯與欄位說明：

| 欄位                | 說明 |
|---------------------|------|
| **Industry**        | 股票原始產業名稱（Yahoo Finance 提供） |
| **Matched Industry**| 模糊比對後找到的對照產業（用於連接真實平均倍數）|
| **P/E**, **P/B**    | 公司目前的實際倍數 |
| **Industry Avg P/E / P/B** | 對照產業的平均倍數（2024年資料）|
| **P/E Undervalued** | 若公司 P/E < 平均，顯示 True |
| **P/B Undervalued** | 若公司 P/B < 平均，顯示 True |

---

### ✅ 如何解讀 Undervalued = True？

| 判斷 | 說明 |
|------|------|
| `P/E Undervalued = True` | 公司本益比低於產業平均，可能存在低估 |
| `P/B Undervalued = True` | 公司股價低於帳面價值，可能存在價值機會 |
| **兩者皆為 True** | 在盈餘與資產上都被市場低估，為**優先關注對象** |

---

### ⚠️ 注意事項：

- 若公司 EPS 為負，P/E 將顯示為 `NaN`，屬於**虧損狀態**
- 在金融業、能源業等資產導向產業，P/B 通常比 P/E 更具參考性
- 相對估值僅是初步工具，建議搭配 DCF / 巴菲特模型進行深入分析

---

### 📘 實例說明（範例表格解讀）

| 股票 | P/E Undervalued | P/B Undervalued | 說明 |
|------|------------------|------------------|------|
| **JPM** | ✅ | ❌ | 銀行股，P/E 相對低估，財報穩定 |
| **INTC** | ❌ | ✅ | 處於虧損階段，無法判斷 P/E，但帳面價值仍低估 |
| **AVGO** | ❌ | ✅ | 科技股帳面價值低，但成長性強，估值高屬合理 |

---

### ✅ 操作建議：

1. 以 `P/E Undervalued + P/B Undervalued = True` 作為初步選股篩選
2. 避免將 EPS 為負的公司納入 P/E 比較（可額外建立 `Loss-Making` 欄位）
3. 可將估值訊號量化成「估值分數」作為綜合排序依據
4. 最終建議搭配 DCF、Buffett 模型進行交叉驗證

---

> ✅ 建議：你可進一步使用條形圖視覺化各公司與產業平均的 P/E 差距，輔助投資決策。


In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
import difflib

# ✅ 自動模糊匹配產業名稱
def match_industry_name(industry_name, benchmark_dict):
    candidates = benchmark_dict.keys()
    match = difflib.get_close_matches(industry_name, candidates, n=1, cutoff=0.6)
    return match[0] if match else None

# ✅ 產業估值基準資料（來源整合自 NYU Stern, Finbox, Macrotrends）
industry_benchmarks = {
    "Consumer Electronics": {"avg_pe": 22.0, "avg_pb": 6.0},
    "Banks - Diversified": {"avg_pe": 13.8, "avg_pb": 1.2},
    "Household & Personal Products": {"avg_pe": 21.4, "avg_pb": 4.5},
    "Semiconductors": {"avg_pe": 22.3, "avg_pb": 6.8},
    "Software - Infrastructure": {"avg_pe": 30.5, "avg_pb": 10.2},
    "Software - Application": {"avg_pe": 32.1, "avg_pb": 9.5},
    "Medical Devices": {"avg_pe": 28.7, "avg_pb": 5.6},
    "Pharmaceuticals": {"avg_pe": 17.5, "avg_pb": 4.1},
    "Biotechnology": {"avg_pe": 21.0, "avg_pb": 5.2},
    "Oil & Gas Integrated": {"avg_pe": 9.1, "avg_pb": 1.6},
    "Utilities - Regulated Electric": {"avg_pe": 16.2, "avg_pb": 2.0},
    "REIT - Retail": {"avg_pe": 14.1, "avg_pb": 1.5},
    "Insurance - Diversified": {"avg_pe": 11.6, "avg_pb": 1.1},
    "Insurance - Life": {"avg_pe": 8.9, "avg_pb": 0.9},
    "Insurance - Property & Casualty": {"avg_pe": 10.4, "avg_pb": 1.3},
    "Retail - Defensive": {"avg_pe": 20.0, "avg_pb": 4.3},
    "Retail - Cyclical": {"avg_pe": 24.5, "avg_pb": 6.1},
    "Restaurants": {"avg_pe": 27.8, "avg_pb": 8.4},
    "Telecom Services": {"avg_pe": 12.9, "avg_pb": 1.8},
    "Aerospace & Defense": {"avg_pe": 19.2, "avg_pb": 3.4},
    "Industrial Products": {"avg_pe": 18.5, "avg_pb": 2.9},
    "Packaging & Containers": {"avg_pe": 17.8, "avg_pb": 2.5},
    "Auto Manufacturers": {"avg_pe": 11.4, "avg_pb": 1.1},
    "Specialty Chemicals": {"avg_pe": 20.3, "avg_pb": 2.7},
    "Gold": {"avg_pe": 16.7, "avg_pb": 1.9},
    "Banks - Regional": {"avg_pe": 11.2, "avg_pb": 1.0},
    "REIT - Industrial": {"avg_pe": 15.8, "avg_pb": 1.8},
    "Healthcare Plans": {"avg_pe": 17.2, "avg_pb": 3.1},
    "Beverages - Non-Alcoholic": {"avg_pe": 23.1, "avg_pb": 5.5},
    "Internet Retail": {"avg_pe": 61.2, "avg_pb": 15.4},
    "Internet Content & Information": {"avg_pe": 38.7, "avg_pb": 10.3},
    "Credit Services": {"avg_pe": 21.5, "avg_pb": 6.7},
    "Discount Stores": {"avg_pe": 24.0, "avg_pb": 5.0}
}


tickers= [
    "AAPL", "MSFT", "PG", "KO", "AMZN", "META", "NVDA", "AVGO", "AXP", "TSLA",
    "JPM", "WMT", "GOOG", "TSM", "INTC", "CRM"
]


# ✅ 建立估值資料結果
relative_results = []

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        pe = info.get("trailingPE", np.nan)
        pb = info.get("priceToBook", np.nan)
        eps = info.get("trailingEps", np.nan)
        industry = info.get("industry", "Unknown")
        sector = info.get("sector", "Unknown")

        matched_industry = match_industry_name(industry, industry_benchmarks)
        industry_avg = industry_benchmarks.get(matched_industry, {"avg_pe": np.nan, "avg_pb": np.nan})

        pe_undervalued = pe < industry_avg["avg_pe"] if not np.isnan(pe) else False
        pb_undervalued = pb < industry_avg["avg_pb"] if not np.isnan(pb) else False
        is_loss_making = eps < 0 if not np.isnan(eps) else False

        relative_results.append({
            "Ticker": ticker,
            "P/E": round(pe, 2) if not np.isnan(pe) else "N/A",
            "P/B": round(pb, 2) if not np.isnan(pb) else "N/A",
            "EPS": round(eps, 2) if not np.isnan(eps) else "N/A",
            "Loss-Making": is_loss_making,
            "Industry": industry,
            "Matched Industry": matched_industry or "N/A",
            "Sector": sector,
            "Industry Avg P/E": industry_avg["avg_pe"],
            "Industry Avg P/B": industry_avg["avg_pb"],
            "P/E Undervalued": pe_undervalued,
            "P/B Undervalued": pb_undervalued
        })
    except Exception as e:
        relative_results.append({
            "Ticker": ticker,
            "Error": str(e)
        })

# ✅ 顯示結果
df_relative = pd.DataFrame(relative_results)
pd.set_option('display.float_format', '{:.2f}'.format)
display(df_relative)


,Ticker,P/E,P/B,EPS,Loss-Making,Industry,Matched Industry,Sector,Industry Avg P/E,Industry Avg P/B,P/E Undervalued,P/B Undervalued
0,AAPL,33.12,47.49,6.41,False,Consumer Electronics,Consumer Electronics,Technology,22.00,6.00,False,False
1,MSFT,34.98,10.46,12.95,False,Software - Infrastructure,Software - Infrastructure,Technology,30.50,10.20,False,False
2,PG,25.08,7.20,6.30,False,Household & Personal Products,Household & Personal Products,Consumer Defensive,21.40,4.50,False,False
3,KO,27.78,11.36,2.49,False,Beverages - Non-Alcoholic,Beverages - Non-Alcoholic,Consumer Defensive,23.10,5.50,False,False
4,AMZN,34.24,7.30,6.14,False,Internet Retail,Internet Retail,Consumer Cyclical,61.20,15.40,True,True
5,META,25.78,8.99,25.58,False,Internet Content & Information,Internet Content & Information,Communication Services,38.70,10.30,True,True
6,NVDA,46.03,41.76,2.94,False,Semiconductors,Semiconductors,Technology,22.30,6.80,False,False
7,AVGO,107.46,3.92,2.16,False,Semiconductors,Semiconductors,Technology,22.30,6.80,False,True
8,AXP,20.97,6.74,14.31,False,Credit Services,Credit Services,Financial Services,21.50,6.70,True,False
9,TSLA,199.82,15.00,1.74,False,Auto Manufacturers,Auto Manufacturers,Consumer Cyclical,11.40,1.10,False,False


# 5. 格雷厄姆數（Graham Number）

---

### 🧠 適用對象：
- 價值投資初學者常用的「保守估值上限」
- Graham Number 適用於「有穩定盈餘與帳面資產的公司」，例如成熟的消費品、製造、金融企業。不適用於成長型科技股或 EPS 為負的新創企業。
- 不適用成長型科技股, 新創
---

### 📌 概念簡介：

格雷厄姆數是由「價值投資之父」班傑明·格雷厄姆所提出，用來評估某支股票的**合理價格上限**。  
如果股票價格高於這個數，可能被高估；若低於，可能有安全邊際。

---

### 📐 公式：

$$
\text{Graham Number} = \sqrt{22.5 \times \text{每股盈餘 (EPS)} \times \text{每股淨值 (BVPS)}}
$$

- \( 22.5 \)：是格雷厄姆建議的 P/E（15） × P/B（1.5）的保守倍數
- \( \text{EPS} \)：每股盈餘（Earnings Per Share）
- \( \text{BVPS} \)：每股淨值（Book Value Per Share）

---

### 🔍 解讀方式：

- 若當前股價 **遠低於** Graham Number ⇒ 可能被低估
- 若當前股價 **接近或高於** Graham Number ⇒ 謹慎評估成長性與風險

---

In [26]:
import yfinance as yf
import pandas as pd
import numpy as np

# ✅ 計算 Graham Number
def graham_number(eps, bvps):
    try:
        return np.sqrt(22.5 * eps * bvps) if eps > 0 and bvps > 0 else np.nan
    except:
        return np.nan

tickers = [
    "KO", "PG", "JNJ",  # 成熟型消費與醫療
    "JPM", "WMT", "T",  # 銀行、零售、電信
    "XOM", "AXP", "INTC"  # 能源、金融、半導體（偏成熟）
]



# ✅ 收集估值結果
graham_results = []

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        eps = info.get("trailingEps", np.nan)
        bvps = info.get("bookValue", np.nan)
        price = info.get("currentPrice", np.nan)
        name = info.get("shortName", ticker)

        graham_val = graham_number(eps, bvps)
        undervalued = price < graham_val if not np.isnan(graham_val) and not np.isnan(price) else False

        graham_results.append({
            "Ticker": ticker,
            "Name": name,
            "EPS": round(eps, 2) if not np.isnan(eps) else "N/A",
            "BVPS": round(bvps, 2) if not np.isnan(bvps) else "N/A",
            "Current Price": round(price, 2) if not np.isnan(price) else "N/A",
            "Graham Number": round(graham_val, 2) if not np.isnan(graham_val) else "N/A",
            "Undervalued (Graham)": "Yes" if undervalued else "No"
        })

    except Exception as e:
        graham_results.append({
            "Ticker": ticker,
            "Name": "Error",
            "EPS": "N/A",
            "BVPS": "N/A",
            "Current Price": "N/A",
            "Graham Number": "N/A",
            "Undervalued (Graham)": f"Error: {str(e)}"
        })

# ✅ 顯示結果
df_graham = pd.DataFrame(graham_results)
pd.set_option('display.float_format', '{:.2f}'.format)
display(df_graham)


,Ticker,Name,EPS,BVPS,Current Price,Graham Number,Undervalued (Graham)
0,KO,Coca-Cola Company (The),2.49,6.09,69.16,18.47,No
1,PG,Procter & Gamble Company (The),6.30,21.96,158.02,55.80,No
2,JNJ,Johnson & Johnson,8.99,32.47,146.36,81.04,No
3,JPM,JP Morgan Chase & Co.,20.37,119.24,265.64,233.77,No
4,WMT,Walmart Inc.,2.41,11.34,96.83,24.80,No
5,T,AT&T Inc.,1.63,14.42,26.44,23.00,No
6,XOM,Exxon Mobil Corporation,7.54,60.96,108.48,101.70,No
7,AXP,American Express Company,14.31,44.51,300.10,119.71,No
8,INTC,Intel Corporation,-4.48,22.87,21.52,N/A,No


# 6. ROE （股東權益報酬率）

---

### 🧠 適用對象：
- 想從「資本回報效率」角度評估企業價值者
- 適合用來預估成長性、作為 DCF 成長率或穩定期估值參考指標


# ✅ 適合使用 ROE 分析的公司類型：

| 類型                     | 為什麼適合                                                | 範例股票              |
|--------------------------|------------------------------------------------------------|-----------------------|
| 成熟型消費品企業         | 盈餘穩定、資本結構清晰，ROE 反映經營效率                  | KO、PG、JNJ、WMT      |
| 金融股（銀行、信用卡）   | 資產槓桿高，ROE 是關鍵指標                                 | JPM、BAC、AXP、GS     |
| 資本密集型企業           | ROE 可以揭露資本使用效率                                   | XOM、CVX、CAT         |
| 大型科技企業（盈利穩定） | 若成長放緩，ROE 可衡量其資本回報力                         | AAPL、MSFT、AVGO      |

---

# ❌ 不適用的公司類型：

| 類型                     | 為什麼不適合                                              | 範例股票              |
|--------------------------|------------------------------------------------------------|-----------------------|
| 早期成長型企業           | 尚未盈利或盈餘不穩定，ROE 失真                              | SNOW、RIVN、PLTR      |
| 帳面資本極小的科技股     | ROE 會因資本基數小而異常高                                 | META、NFLX（特定年份） |
| ROE 為負或異常值的公司   | 可能代表虧損，資本結構扭曲                                 | INTC、NIO（虧損期）    |

---

# 🧠 判讀指標參考（可加進 Markdown）

| ROE 區間         | 解釋與建議                                                      |
|------------------|-----------------------------------------------------------------|
| > 20%            | 資本效率極佳，有護城河或壟斷力（如 AAPL、MSFT）                 |
| 10–20%           | 普通優秀企業，穩健型投資可接受                                  |
| < 10%            | 資本使用效率偏低，可能需避開或搭配其他分析指標                  |
| < 0% 或異常值    | 公司處於虧損，或資本結構異常，不適合單獨用 ROE 評估              |



---

### 📌 概念簡介：

ROE（Return on Equity）是衡量公司利用股東資本創造淨利的能力。  
它是成長股分析與「剩餘收益估值模型（Residual Income Model）」的核心指標之一。

---

### 📐 公式：

#### 一般定義：

$$
\text{ROE} = \frac{\text{淨利}}{\text{股東權益}}
$$

---

### 📌 延伸應用：永續成長率估算

若企業保持固定的盈餘保留率 \( b \)，則理論上的內部成長率 \( g \) 為：

$$
g = \text{ROE} \times b
$$

- \( b \)：盈餘保留率（= 1 - 配息率）
- \( g \)：可用於 DCF 或 DDM 的成長率估算依據

---


In [27]:
import yfinance as yf
import pandas as pd
import numpy as np

# ✅ 抓取 ROE 與公司產業資料
def get_roe_info(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        roe = info.get("returnOnEquity", None)
        name = info.get("shortName", ticker)
        industry = info.get("industry", "N/A")
        sector = info.get("sector", "N/A")

        return {
            "Ticker": ticker,
            "Name": name,
            "Sector": sector,
            "Industry": industry,
            "ROE (%)": round(roe * 100, 2) if roe is not None else "N/A"
        }
    except Exception as e:
        return {
            "Ticker": ticker,
            "Name": "Error",
            "Sector": "N/A",
            "Industry": "N/A",
            "ROE (%)": f"Error: {str(e)}"
        }



tickers = ["AAPL", "MSFT", "KO", "PG", "JNJ", "AXP", "JPM", "WMT", "XOM"]


# ✅ 批次處理
roe_results = [get_roe_info(tkr) for tkr in tickers]
df_roe = pd.DataFrame(roe_results)
df_roe["ROE_num"] = pd.to_numeric(df_roe["ROE (%)"], errors="coerce")

# 定義分類函數
def classify_roe(roe):
    if pd.isna(roe):
        return "N/A"
    elif roe >= 20:
        return "資本效率極佳"
    elif roe >= 10:
        return "普通優秀企業"
    elif roe >= 0:
        return "資本效率偏低"
    else:
        return "虧損或異常值"

# 套用分類邏輯
df_roe["ROE 評估等級"] = df_roe["ROE_num"].apply(classify_roe)


# ✅ 排序（排除非數值欄位）
df_roe_sorted = df_roe.copy()
df_roe_sorted["ROE (%) (sort)"] = pd.to_numeric(df_roe["ROE (%)"], errors='coerce')
df_roe_sorted = df_roe_sorted.sort_values(by="ROE (%) (sort)", ascending=False).drop(columns="ROE (%) (sort)").reset_index(drop=True)

# ✅ 顯示表格
pd.set_option('display.float_format', '{:.2f}'.format)
df_roe = df_roe_sorted 
display(df_roe_sorted)


,Ticker,Name,Sector,Industry,ROE (%),ROE_num,ROE 評估等級
0,AAPL,Apple Inc.,Technology,Consumer Electronics,138.01,138.01,資本效率極佳
1,KO,Coca-Cola Company (The),Consumer Defensive,Beverages - Non-Alcoholic,38.78,38.78,資本效率極佳
2,AXP,American Express Company,Financial Services,Credit Services,34.27,34.27,資本效率極佳
3,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,33.61,33.61,資本效率極佳
4,PG,Procter & Gamble Company (The),Consumer Defensive,Household & Personal Products,30.29,30.29,資本效率極佳
5,JNJ,Johnson & Johnson,Healthcare,Drug Manufacturers - General,29.45,29.45,資本效率極佳
6,WMT,Walmart Inc.,Consumer Defensive,Discount Stores,21.41,21.41,資本效率極佳
7,JPM,JP Morgan Chase & Co.,Financial Services,Banks - Diversified,17.35,17.35,普通優秀企業
8,XOM,Exxon Mobil Corporation,Energy,Oil & Gas Integrated,14.30,14.30,普通優秀企業


# 7. EV / EBITDA 估值模型

### 📘 EV/EBITDA 模型簡介：

- **EV/EBITDA** 是評估企業估值的重要指標，特別適用於跨公司或產業比較
- 常見合理範圍為 **6 到 15**
    - < 6：可能被低估（或為低成長企業）
    - > 15：可能被高估（或為高成長、科技股）

- 適用產業：製造、零售、能源、成熟科技（如 AAPL、XOM、WMT）
- 不適用：尚未穩定獲利或 EBITDA 為負的成長股（如 PLTR、SNOW）

---


### 🧠 適用對象：
- 適合評估不同規模、資本結構不同的企業價值
- 廣泛應用於併購（M&A）、企業價值比較（尤其是非美股）


---

### 📌 概念介紹：

- **EV（Enterprise Value）**：企業總價值，考慮市值、債務、現金
- **EBITDA**：稅息折舊攤銷前盈餘，反映公司營運獲利能力

---

### 📐 模型公式：

$$
\text{EV/EBITDA} = \frac{\text{企業價值（EV）}}{\text{稅息折舊攤銷前盈餘（EBITDA）}}
$$

其中，

$$
\text{EV} = \text{市值} + \text{總負債} - \text{現金}
$$

---

### 🔍 解讀方式：

| 倍數範圍 | 評價 |
|----------|------|
| < 8 倍   | 可能被低估（便宜） |
| 8–12 倍 | 合理區間（產業視角） |
| > 12 倍 | 可能被高估，需看成長性 |

⚠️ 不同行業標準不同，例如科技股常高於 15x，公用事業可能 < 8x。



In [29]:
import yfinance as yf
import pandas as pd
import numpy as np

def calculate_ev_ebitda(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        name = info.get("shortName", ticker)
        sector = info.get("sector", "N/A")
        industry = info.get("industry", "N/A")

        market_cap = info.get("marketCap", np.nan)
        debt = info.get("totalDebt", 0)
        cash = info.get("totalCash", 0)
        ebitda = info.get("ebitda", np.nan)

        # 檢查是否可計算 EV/EBITDA
        if all([not np.isnan(val) for val in [market_cap, ebitda]]) and ebitda != 0:
            ev = market_cap + debt - cash
            ev_ebitda = ev / ebitda

            # 標記是否屬於合理區間（常見合理值為 6–15）
            if ev_ebitda < 6:
                tag = "可能低估"
            elif ev_ebitda > 15:
                tag = "可能高估"
            else:
                tag = "合理區間"

            return {
                "Ticker": ticker,
                "Name": name,
                "Sector": sector,
                "Industry": industry,
                "Market Cap (B)": round(market_cap / 1e9, 2),
                "EBITDA (B)": round(ebitda / 1e9, 2),
                "EV/EBITDA": round(ev_ebitda, 2),
                "估值評估": tag
            }
        else:
            return {
                "Ticker": ticker,
                "Name": name,
                "Sector": sector,
                "Industry": industry,
                "Market Cap (B)": "N/A",
                "EBITDA (B)": "N/A",
                "EV/EBITDA": "N/A",
                "估值評估": "N/A"
            }

    except Exception as e:
        return {
            "Ticker": ticker,
            "Name": "Error",
            "Sector": "N/A",
            "Industry": "N/A",
            "Market Cap (B)": "N/A",
            "EBITDA (B)": "N/A",
            "EV/EBITDA": "N/A",
            "估值評估": str(e)}

            
# 股票清單
tickers = ["AAPL", "MSFT", "KO", "JNJ", "T", "WMT", "XOM", "JPM", "META"]


# 執行估值計算
ev_ebitda_results = [calculate_ev_ebitda(tkr) for tkr in tickers]
df_ev = pd.DataFrame(ev_ebitda_results)
df_ev["EV/EBITDA_num"] = pd.to_numeric(df_ev["EV/EBITDA"], errors="coerce")


def classify_ev_ebitda(ev_val):
    if pd.isna(ev_val):
        return "N/A"
    elif ev_val < 6:
        return "低估（可能低成長）"
    elif ev_val > 15:
        return "高估（可能高成長）"
    else:
        return "合理估值區間"


df_ev["EV/EBITDA 區間評價"] = df_ev["EV/EBITDA_num"].apply(classify_ev_ebitda)
df_ev.drop(columns=["EV/EBITDA_num"], inplace=True)
pd.set_option('display.float_format', '{:.2f}'.format)
display(df_ev)

,Ticker,Name,Sector,Industry,Market Cap (B),EBITDA (B),EV/EBITDA,估值評估,EV/EBITDA 區間評價
0,AAPL,Apple Inc.,Technology,Consumer Electronics,3171.32,138.87,23.20,可能高估,高估（可能高成長）
1,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,3366.49,149.17,22.74,可能高估,高估（可能高成長）
2,KO,Coca-Cola Company (The),Consumer Defensive,Beverages - Non-Alcoholic,297.68,15.39,21.70,可能高估,高估（可能高成長）
3,JNJ,Johnson & Johnson,Healthcare,Drug Manufacturers - General,352.15,30.03,12.18,合理區間,合理估值區間
4,T,AT&T Inc.,Communication Services,Telecom Services,190.25,44.26,7.42,合理區間,合理估值區間
5,WMT,Walmart Inc.,Consumer Defensive,Discount Stores,774.73,42.22,19.61,可能高估,高估（可能高成長）
6,XOM,Exxon Mobil Corporation,Energy,Oil & Gas Integrated,467.51,64.87,7.52,合理區間,合理估值區間
7,JPM,JP Morgan Chase & Co.,Financial Services,Banks - Diversified,N/A,N/A,N/A,N/A,N/A
8,META,"Meta Platforms, Inc.",Communication Services,Internet Content & Information,1657.85,87.98,18.61,可能高估,高估（可能高成長）


#  Conclusion







In [42]:
import pandas as pd
import numpy as np

# 安全取得模型欄位值
def safe_get(df, ticker, column):
    try:
        return df[df["Ticker"] == ticker][column].values[0]
    except:
        return "不適用"

# 整合函式
def collect_model_estimates(df_dcf, df_ddm, df_buffett, df_graham, df_relative, df_roe, df_ev):
    # 整合所有股票
    tickers = sorted(set(
        df_dcf["Ticker"]) |
        set(df_ddm["Ticker"]) |
        set(df_buffett["Ticker"]) |
        set(df_graham["Ticker"]) |
        set(df_relative["Ticker"]) |
        set(df_roe["Ticker"]) |
        set(df_ev["Ticker"])
    )

    summary = []

    for ticker in tickers:
        row = {"Ticker": ticker}

        # DCF
        note = safe_get(df_dcf, ticker, "Note")
        row["DCF"] = "低估" if note == "Undervalued" else "高估" if note == "Overvalued" else "不適用"

        # DDM
        note = safe_get(df_ddm, ticker, "Note")
        row["DDM"] = "低估" if note == "Undervalued" else "高估" if note == "Overvalued" else "不適用"

        # Buffett
        val = safe_get(df_buffett, ticker, "Undervalued (Buffett)")
        row["Buffett"] = "低估" if val == "Yes" else "高估" if val == "No" else "不適用"

        # Graham Number
        val = safe_get(df_graham, ticker, "Undervalued (Graham)")
        row["Graham Number"] = "低估" if val == "Yes" else "高估" if val == "No" else "不適用"

        # P/E, P/B
        pe = safe_get(df_relative, ticker, "P/E Undervalued")
        pb = safe_get(df_relative, ticker, "P/B Undervalued")
        if pd.isna(pe) and pd.isna(pb):
            row["P/E、P/B 相對估值"] = "不適用"
        elif pe or pb:
            row["P/E、P/B 相對估值"] = "低估"
        else:
            row["P/E、P/B 相對估值"] = "高估"

        # ROE 評估模型
        row["ROE 評估模型"] = safe_get(df_roe, ticker, "ROE 評估等級")

        # EV/EBITDA 倍數法
        row["EV/EBITDA 倍數法"] = safe_get(df_ev, ticker, "估值評估")

        summary.append(row)

    return pd.DataFrame(summary)

# ⬇️ 使用範例
df_summary = collect_model_estimates(df_dcf, df_ddm, df_buffett, df_graham, df_relative, df_roe, df_ev)


# 分數對應表
score_map = {
    "低估": 1,
    "高估": -1,
    "可能高估": -1,
    "資本效率極佳": 1,
    "普通優秀企業": 0,
    "合理區間": 0,
    "不適用": 0,
    np.nan: 0
}

# 需要評分的模型欄位
model_cols = [
    "DCF", "DDM", "Buffett", "Graham Number",
    "P/E、P/B 相對估值", "ROE 評估模型", "EV/EBITDA 倍數法"
]

# 計算每一欄的分數
for col in model_cols:
    df_summary[col + "_分數"] = df_summary[col].map(score_map).fillna(0)

# 加總有效模型數（只排除 "不適用" 與 NaN）
df_summary["有效模型數"] = df_summary[model_cols].apply(
    lambda row: sum([1 for v in row if v not in ["不適用", np.nan]]), axis=1
)

# 加總總得分
df_summary["估值總得分"] = df_summary[[col + "_分數" for col in model_cols]].sum(axis=1)

# 計算最終標準化得分（-1～+1）
df_summary["Score"] = (df_summary["估值總得分"] / df_summary["有效模型數"]).round(2)

# 👉 清理中間欄位（可選）
df_summary.drop(columns=[col + "_分數" for col in model_cols] + ["估值總得分"], inplace=True)

# ✅ 顯示最終結果
display(df_summary.sort_values("Score", ascending=False).reset_index(drop=True))



,Ticker,DCF,DDM,Buffett,Graham Number,P/E、P/B 相對估值,ROE 評估模型,EV/EBITDA 倍數法,有效模型數,Score
0,AMZN,不適用,不適用,不適用,不適用,低估,不適用,不適用,1,1.00
1,AVGO,不適用,不適用,不適用,不適用,低估,不適用,不適用,1,1.00
2,CRM,不適用,不適用,不適用,不適用,低估,不適用,不適用,1,1.00
3,GOOG,低估,不適用,不適用,不適用,低估,不適用,不適用,2,1.00
4,TMUS,不適用,低估,不適用,不適用,低估,不適用,不適用,2,1.00
5,XOM,低估,低估,不適用,高估,低估,普通優秀企業,合理區間,6,0.33
6,TSM,低估,不適用,高估,不適用,低估,不適用,不適用,3,0.33
7,T,不適用,低估,不適用,高估,低估,不適用,合理區間,4,0.25
8,META,不適用,不適用,不適用,不適用,低估,不適用,可能高估,2,0.00
9,JNJ,高估,低估,高估,高估,低估,資本效率極佳,合理區間,7,0.00


In [ ]:

# === Model: Relative Valuation (P/E & P/B) ===
import os, json
import pandas as pd
import yfinance as yf
import numpy as np

# Ensure OUTPUT_DIR exists (fallback)
try:
    OUTPUT_DIR
except NameError:
    OUTPUT_DIR = os.path.join(os.path.abspath("."), "output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Ensure TICKERS exists (fallback: read from tickers.txt if present)
try:
    TICKERS
except NameError:
    tickers_path = os.path.join(os.path.abspath("."), "tickers.txt")
    if os.path.exists(tickers_path):
        with open(tickers_path, "r", encoding="utf-8") as f:
            TICKERS = [t.strip() for t in f if t.strip()]
    else:
        TICKERS = []

def pe_pb_for_ticker(ticker):
    t = yf.Ticker(ticker)
    info = getattr(t, "info", {}) or {}
    price = info.get("currentPrice") or info.get("regularMarketPrice")
    pe = info.get("trailingPE")
    pb = info.get("priceToBook")
    eps = info.get("trailingEps")
    bvps = info.get("bookValue")

    return {
        "Ticker": ticker,
        "Price": price,
        "TrailingPE": pe,
        "PriceToBook": pb,
        "EPS": eps,
        "BVPS": bvps
    }

pe_pb_rows = [pe_pb_for_ticker(t) for t in TICKERS]
pe_pb_df = pd.DataFrame(pe_pb_rows)

# Save standalone CSV
pe_pb_path = os.path.join(OUTPUT_DIR, "pe_pb.csv")
pe_pb_df.to_csv(pe_pb_path, index=False)
print("Saved:", pe_pb_path)

pe_pb_df


In [ ]:

# === Model: Residual Income (ROE-based steady-state) ===
import os, json, math
import pandas as pd
import yfinance as yf
import numpy as np

# Ensure config & OUTPUT_DIR
try:
    CFG
except NameError:
    config_path = os.path.join(os.path.abspath("."), "config.json")
    with open(config_path, "r", encoding="utf-8") as f:
        CFG = json.load(f)

try:
    OUTPUT_DIR
except NameError:
    OUTPUT_DIR = os.path.join(os.path.abspath("."), "output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Ensure TICKERS
try:
    TICKERS
except NameError:
    tickers_path = os.path.join(os.path.abspath("."), "tickers.txt")
    with open(tickers_path, "r", encoding="utf-8") as f:
        TICKERS = [t.strip() for t in f if t.strip()]

# Parameters
r_coe = float(CFG.get("discount_rate", 0.09))  # cost of equity proxy
payout_ratio_default = float(CFG.get("ddm_defaults", {}).get("payout_ratio", 0.4))

def residual_income_value_per_share(ticker, r=None, payout_ratio=None):
    # Steady-state Residual Income model (per share):
    # V0 = BVPS + (ROE - r) * BVPS / (r - g), with g = ROE * retention
    # Preconditions: r > g
    # Returns: fair value per share or None
    r = float(r if r is not None else r_coe)
    payout_ratio = float(payout_ratio if payout_ratio is not None else payout_ratio_default)

    t = yf.Ticker(ticker)
    info = getattr(t, "info", {}) or {}
    roe = info.get("returnOnEquity")  # decimal
    bvps = info.get("bookValue")
    price = info.get("currentPrice") or info.get("regularMarketPrice")

    if roe is None or bvps is None:
        return None, price, roe, bvps, None, r, payout_ratio

    try:
        roe = float(roe)
        bvps = float(bvps)
    except Exception:
        return None, price, roe, bvps, None, r, payout_ratio

    retention = max(0.0, min(1.0, 1.0 - payout_ratio))
    g = roe * retention  # implied BVPS growth
    if r <= g or bvps <= 0:
        return None, price, roe, bvps, g, r, payout_ratio

    fair = bvps + (roe - r) * bvps / (r - g)
    return float(fair), price, roe, bvps, g, r, payout_ratio

ri_rows = []
for tk in TICKERS:
    fair, price, roe, bvps, g, r, pr = residual_income_value_per_share(tk)
    ri_rows.append({
        "Ticker": tk,
        "Price": price,
        "ROE": roe,
        "BVPS": bvps,
        "Retention": None if pr is None else (1.0 - pr),
        "g_implied": g,
        "r_cost_of_equity": r,
        "RI_FairValue_ps": fair
    })

ri_df = pd.DataFrame(ri_rows)
ri_df_path = os.path.join(OUTPUT_DIR, "residual_income.csv")
ri_df.to_csv(ri_df_path, index=False)
print("Saved:", ri_df_path)

ri_df


In [ ]:

# === Model: EV/EBITDA Multiple ===
import os, json
import pandas as pd
import yfinance as yf
import numpy as np

# Ensure OUTPUT_DIR and TICKERS
try:
    OUTPUT_DIR
except NameError:
    OUTPUT_DIR = os.path.join(os.path.abspath("."), "output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

try:
    TICKERS
except NameError:
    tickers_path = os.path.join(os.path.abspath("."), "tickers.txt")
    with open(tickers_path, "r", encoding="utf-8") as f:
        TICKERS = [t.strip() for t in f if t.strip()]

def ev_ebitda_for_ticker(ticker):
    t = yf.Ticker(ticker)
    info = getattr(t, "info", {}) or {}

    price = info.get("currentPrice") or info.get("regularMarketPrice")
    shares = info.get("sharesOutstanding")
    debt = info.get("totalDebt")
    cash = info.get("totalCash")
    ebitda = info.get("ebitda")
    ev = info.get("enterpriseValue")

    # Fallback EV if missing
    if ev is None:
        if price is not None and shares is not None and debt is not None and cash is not None:
            try:
                ev = float(price) * float(shares) + float(debt) - float(cash)
            except Exception:
                ev = None

    multiple = None
    try:
        if ev is not None and ebitda not in (None, 0):
            multiple = float(ev) / float(ebitda)
    except Exception:
        multiple = None

    return {
        "Ticker": ticker,
        "Price": price,
        "SharesOut": shares,
        "Debt": debt,
        "Cash": cash,
        "EBITDA": ebitda,
        "EnterpriseValue": ev,
        "EV_EBITDA": multiple
    }

ev_rows = [ev_ebitda_for_ticker(t) for t in TICKERS]
ev_df = pd.DataFrame(ev_rows)

ev_path = os.path.join(OUTPUT_DIR, "ev_ebitda.csv")
ev_df.to_csv(ev_path, index=False)
print("Saved:", ev_path)

ev_df


In [ ]:

# === Aggregate all model outputs into one comparison CSV ===
import os
import pandas as pd

OUTPUT_DIR = OUTPUT_DIR if 'OUTPUT_DIR' in globals() else os.path.join(os.path.abspath("."), "output")

# Expected individual files (some produced earlier in your notebook)
files = {
    "core_models": os.path.join(OUTPUT_DIR, "results.csv"),          # from DCF/DDM/Graham/Buffett
    "pe_pb": os.path.join(OUTPUT_DIR, "pe_pb.csv"),
    "residual_income": os.path.join(OUTPUT_DIR, "residual_income.csv"),
    "ev_ebitda": os.path.join(OUTPUT_DIR, "ev_ebitda.csv")
}

dfs = {}
for k, p in files.items():
    if os.path.exists(p):
        dfs[k] = pd.read_csv(p)
    else:
        print(f"⚠️ Missing file: {p}")

# Merge on Ticker (left join on core_models if present)
if "core_models" in dfs:
    merged = dfs["core_models"].copy()
    for k in ["pe_pb", "residual_income", "ev_ebitda"]:
        if k in dfs:
            keep_cols = [c for c in dfs[k].columns if c != "Ticker"]
            merged = merged.merge(dfs[k][["Ticker"] + keep_cols], on="Ticker", how="left")
else:
    # If core_models not present, merge everything available
    available = list(dfs.values())
    if available:
        merged = available[0]
        for d in available[1:]:
            keep_cols = [c for c in d.columns if c != "Ticker"]
            merged = merged.merge(d[["Ticker"] + keep_cols], on="Ticker", how="outer")
    else:
        merged = pd.DataFrame()

# Save
combined_path = os.path.join(OUTPUT_DIR, "combined_models.csv")
merged.to_csv(combined_path, index=False)
print("Saved:", combined_path)

merged.head()
